In [22]:
import math
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

### 读取数据

In [23]:
iris = load_iris()

In [24]:
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target
df.columns = ['speal length', 'speal width', 'petal length', 'petal width', 'label']
df.head()

,speal length,speal width,petal length,petal width,label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [25]:
df.shape

(150, 5)

In [26]:
df.label.value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

### ID3/C4.5算法实现

In [27]:
# 划分训练集测试集
from sklearn.model_selection import train_test_split
data = np.array(df)
X, y = data[:, :-1], data[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [28]:
# 定义计算熵公式的函数
def calcEntropy(target):
    label = np.unique(target)
    n = label.size
    count = np.zeros(n)
    p_i = np.zeros(n)
    for i in range(n):
        count[i] = target[target == label[i]].size
    
    # 计算每个类别的概率
    p_i = np.divide(count, target.size)
    
    # 计算熵
    entropy = 0
    for i in range(n):
        entropy = entropy - p_i[i] * np.log2(p_i[i])
    
    return entropy

In [44]:
ent_dataset = calcEntropy(y_train)
ent_dataset

1.582879624786234

In [103]:
# 定义二分类条件熵公式的函数，把属性的值分为两类
# 对于花瓣宽度来说: 一类是大于 0.8， 另一类就是小于等于 0.8
def calcConditionEntropy(feature, condition, target, ways = 'ID3'):
    true_condition = condition(feature)
    false_condition = true_condition == False
    target_true = target[true_condition]
    target_false = target[false_condition]
    # 每种属性类别的数量除以总数就计算出其概率
    p_true = target_true.size / target.size
    p_false = 1 - p_true
    # 每种属性类别的概率乘以该类别下的信息熵
    entropy = p_true * calcEntropy(target_true) + p_false * calcEntropy(target_false)
    # 信息增益
    entropy = ent_dataset - entropy
    if ways == 'C4.5':
        H = calcEntropy(true_condition)
        if(H != 0):
            entropy = entropy / H
    return entropy

In [109]:
petal_width = X_train[:, 3]
HC = calcConditionEntropy(petal_width, lambda feature:feature<0.8, y_train, 'C4.5')
HC

0.99999999999999989

In [86]:
def generate_feature_points(feature, target):
    """
    生成特征的所有分界点: 先对特征进行排序，然后将 target 有变动的地方作为分界点
    :param feature: 一维数组，一个特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :return: 包含所有分界点的一维数组
    """
    argsort = feature.argsort()
    f1 = feature[argsort]
    t1 = target[argsort]
    
    last_value = target[0]
    split_value = []
    
    # 找出所有分裂点
    for i in range(t1.size):
        if last_value != t1[i]:
            split_value.append((f1[i] + f1[i - 1]) / 2)
            last_value = t1[i]

    return np.array(split_value)

In [111]:
def calc_feature_entropy(feature, target, ways = 'ID3'):
    """
    计算一个特征的所有分界点的条件熵，返回最小的那个条件熵（条件熵越小，信息增益越大）
    :param feature: 一维数组，一个特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :return: 分界点和条件熵
    """
    max_ent_gain = 0
    max_point = 0
    
    points = generate_feature_points(feature, target)
    for p in points:
        entropy = calcConditionEntropy(feature, lambda f: f < p, target, ways)
        if entropy > max_ent_gain:
            max_ent_gain = entropy
            max_point = p

    '没有分界点说明只有一类数据标签，熵为0'
    if points.size == 0:
        max_entropy = ent_dataset

    return max_point, max_ent_gain

In [112]:
calc_feature_entropy(X_train[:, 3], y_train, 'C4.5')

(0.80000000000000004, 0.99999999999999989)

In [113]:
def select_feature(features, target, ways = 'ID3'):
    """
    从所有特征中选择出条件熵最小的特征（即最大增益）
    :param features: 二维数据，包含所有特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :return: 特征索引，条件熵，特征分界点
    """
    max_ent_gain = 0
    max_point = 0
    num = features.shape[1]
    index = 0
    for i in range(num):
        point, entropy = calc_feature_entropy(features[:, i], target, ways)
        if entropy > max_ent_gain:
            index = i
            max_point = point
            max_ent_gain = entropy

    return index, max_point, max_ent_gain  

In [114]:
select_feature(X_train, y_train, 'C4.5')

(2, 2.4500000000000002, 0.99999999999999989)

In [115]:
class TreeNode:
    idn = 0
    feature_index = ''
    feature_point = 0
    feature_entropy = 0
    target_label = ''
    true_node = None
    false_node = None

    @staticmethod
    def decision(feature, point):
        return feature < point

In [128]:
def build_tree(features, target, idn, ways = 'ID3'):
    """
    递归构建决策树
    :param features: 二维数据，包含所有特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :param idn: 决策树节点 id，通过 id 观察决策树计算过程
    :return: 决策树根节点
    """
    node = TreeNode()
    
    '1,若D中实例属于同一类Ck，则T为单节点树，并将类Ck作为结点的类标记，返回T'
    if np.sum(target == target[0]) == len(target):
        node.target_label = target[0]
        return node
    
    ' 若A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T'
    if len(features) == 0:
        node.target_label = target[np.argmax(np.bincount(target))]
        return node

    '选择信息增益最大的特征'
    index, point, entropy = select_feature(features, target, ways)
    node.idn = idn
    node.feature_index = index
    node.feature_point = point
    node.feature_entropy = entropy
    '取出现次数最多的标签作为该特征节点的输出'
    node.target_label = target[np.argmax(np.bincount(target))]

#     print('build tree node id %d, index %d, point %f, entropy %f, label %s ' %
#           (idn, index, point, entropy, node.target_label))

    '熵小于 0.1 时则结束创建子节点，防止过拟合'
    if entropy < 0.1:
#         print('too low entropy : ', entropy)
        return node

    f_copy = features.copy()
    t_copy = target.copy()
    f = f_copy[:, index]
    selector = node.decision(f, point)

    '创建左右两个子节点'
    idn = idn + 1
    node.true_node = build_tree(f_copy[selector, :], t_copy[selector], idn, ways)
    idn = node.true_node.idn + 1
    node.false_node = build_tree(f_copy[selector == False], t_copy[selector == False], idn, ways)
    return node

In [129]:
def predict(X_test, y_test, node):
    score = 0
    length = len(y_test)
    for i in range(length):
        p = node
        while p.true_node or p.false_node:
            if X_test[i][p.feature_index] < p.feature_point:
                p = p.true_node
            else:
                p = p.false_node
        if p.target_label == y_test[i]:
            score += 1
    return score/float(length)

In [130]:
node = build_tree(X_train, y_train, 1)

In [131]:
predict(X_test, y_test, node)

0.9666666666666667

In [134]:
node2 = build_tree(X_train, y_train, 1, 'C4.5')

In [135]:
predict(X_test, y_test, node2)

0.9333333333333333